# Java Threads and Concurrency Utilities

## Chapter - 2
## Synchronization

In [1]:
Runtime.getRuntime().availableProcessors()

4

### The Problems with Threads

* Race Conditions
* Data Races
* Cached Variables

#### Race Condition

A race condition occurs when the correctness of a computation depends on the relative timing or interleaving of multiple threads by the scheduler.
<pre class="brush:java"> 
if (a == 10.0) 
    b = a / 2.0;
</pre>
There is no problem with this code fragment in a single-threaded context, and there is no problem in a multithreaded context when a and b are local variables. 

However, assume that a and b identify instance or class (static) field variables and that two threads simultaneously access this code, and we are implementing this in multithreaded, this will definately be a disater in this case.

* The code fragment is an example of a common type of race condition that’s known as **check-then-act**, in which a potentially stale observation is used to decide on what to do next. 
* Another type of race condition is **read-modify-write**, in which new state is derived from previous state. The previous state is read, then modified, and finally updated to reflect the modified result via three indivisible operations. However, the combination of these operations isn’t indivisible.

#### Data Races

A race condition is often confused with a data race in which two or more threads (in a single application) access the same memory location concurrently, at least one of the accesses is for writing, and these threads don’t coordinate their accesses to that memory. When these conditions hold, access order is non-deterministic. Different results may be generated from run to run, depending on that order. Consider the following example:

<pre class="brush:java"> 
private static Parser parser;
public static Parser getInstance()
{
if (parser == null) parser = new Parser();
   return parser;
}
</pre>

#### Cached Variables
To boost performance, the compiler, the Java virtual machine (JVM), and the operating system can collaborate to cache a variable in a register or a processor-local cache, rather than rely on main memory. Each thread has its own copy of the variable. When one thread writes to this variable, it’s writing to its copy; other threads are unlikely to see the update in their copies.

#### Synchronizing Access to Critical Sections
You can use synchronization to solve the previous thread problems. Synchronization is a JVM feature that ensures that two or more concurrent threads don’t simultaneously execute a critical section, which is a code section that must be accessed in a serial (one thread at a time) manner.

#### Using Synchronized Methods
A synchronized method includes the synchronized keyword in its header. For example, you can use this keyword to synchronize the former getID() method and overcome its read-modify-write race condition as follows:

In [2]:
// public synchronized int getID()
// {
//    return counter++;
// }

When synchronizing on an instance method, the lock is associated with the object on which the method is called. For example, consider the following ID class:


In [3]:
import java.util.Date;
public class ID {
    private int counter; // initialized to 0 by default

    public synchronized int getID() {
        try {
            Thread.sleep(2000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.print(new Date()+"\n");
        return counter++;
    }
}

Suppose you specify the following code sequence

The lock is associated with the ID object whose reference is stored in id. If another thread called id.getID() while this method was executing, the other thread would have to wait until the executing thread released the lock.

In [4]:

ID id = new ID();
// id.getID();
Runnable getID = () -> {
    System.out.println(id.getID());
};

Thread t1 = new Thread(getID);
Thread t2 = new Thread(getID);

t1.start();
t2.start();

t1.join();
t2.join();

Sun Jul 05 21:13:03 IST 2020
0
Sun Jul 05 21:13:05 IST 2020
1


In [5]:
public class IDStatic {
    private static int counter; // initialized to 0 by default

    public static synchronized int getID() {
        try {
            Thread.sleep(2000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.print(new Date() + "\n");
        return counter++;
    }
}

In [6]:
IDStatic id = new IDStatic();
Runnable getID = () -> {
    System.out.println(IDStatic.getID());
};

Thread t1 = new Thread(getID);
Thread t2 = new Thread(getID);

t1.start();
t2.start();

t1.join();
t2.join();

Sun Jul 05 21:13:07 IST 2020
0
Sun Jul 05 21:13:09 IST 2020
1


#### Using Synchronized Blocks

A synchronized block of statements is prefixed by a header that identifies the object whose lock is to be acquired. It has the following syntax:
<pre class="brush:java">
synchronized(object) {
   /* statements */
}
</pre>

According to this syntax, object is an arbitrary object reference. The lock is associated with this object.

#### Beware of Liveness Problems

The term liveness refers to something beneficial happening eventually. A liveness failure occurs when an application reaches a state in which it can make no further progress.
* In a single-threaded application 
    * infinite loop 
* Multithreaded applications face the additional liveness challenges of 
    * deadlock 
    * livelock 
    * starvation

#### Deadlock 
This pathological problem occurs because of too much synchronization via the synchronized keyword. If you’re not careful, you might encounter a situation where locks are acquired by multiple threads, neither thread holds its own lock but holds the lock needed by some other thread, and neither thread can enter and later exit its critical section to release its held lock because another thread holds the lock to that critical section

In [7]:
public class DeadlockDemo {
    private final Object lock1 = new Object();
    private final Object lock2 = new Object();
    
    public void instanceMethod1() {
        synchronized (lock1) {
            synchronized (lock2) {
                System.out.println("first thread in instanceMethod1");
                // critical section guarded first by
                // lock1 and then by lock2
            }
        }
    }

    public void instanceMethod2() {
        synchronized (lock2) {
            synchronized (lock1) {
                System.out.println("second thread in instanceMethod2");
                // critical section guarded first by lock2 and then by lock1
            }
        }
    }
}

In [ ]:

final DeadlockDemo dld = new DeadlockDemo();
Runnable r1 = () -> {
    while (true) {
        dld.instanceMethod1();
        try {
            Thread.sleep(50);
        } catch (InterruptedException ie) {
        }
    }
};

Thread thdA = new Thread(r1);
Runnable r2 = () -> {
    while (true) {
        dld.instanceMethod2();
        try {
            Thread.sleep(50);
        } catch (InterruptedException ie) {
        }
    }
};

Thread thdB = new Thread(r2);
thdA.start();
thdB.start();

first thread in instanceMethod1
first thread in instanceMethod1
first thread in instanceMethod1
second thread in instanceMethod2


#### Consider the following execution sequence:
1. Thread A calls instanceMethod1(), obtains the lock assigned to the lock1-referenced object, and enters its outer critical section (but has not yet acquired the lock assigned to the lock2-referenced object).
2. Thread B calls instanceMethod2(), obtains the lock assigned to the lock2-referenced object, and enters its outer critical section (but has not yet acquired the lock assigned to the lock1-referenced object).
3. Thread A attempts to acquire the lock associated with lock2. The JVM forces the thread to wait outside of the inner critical section because thread B holds that lock.
4. Thread B attempts to acquire the lock associated with lock1. The JVM forces the thread to wait outside of the inner critical section because thread A holds that lock.
5. Neither thread can proceed because the other thread holds the needed lock. You have a deadlock situation and the program (at least in the context of the two threads) freezes up.

__Neither the Java language nor the JVM provides a way to prevent deadlock, and so the burden falls on you. the simplest way to prevent deadlock is to avoid having either a synchronized method or a synchronized block call another synchronized method/block. although this advice prevents deadlock from happening, it’s impractical because one of your synchronized methods/blocks might need to call a synchronized method in a Java api, and the advice is overkill because the synchronized method/block being called might not call any other synchronized method/block, so deadlock would not occur.__

#### Stopping a thread


In [9]:
class StoppableThread extends Thread {
     private volatile boolean stopped; // defaults to false

     @Override
     public void run() {
         while (!stopped)
             System.out.println("running");
     }

     void stopThread() {
         stopped = true;
     }
 }

In [10]:
StoppableThread thd = new StoppableThread();
thd.start();

try {
    Thread.sleep(1); // sleep for 1 second
} catch (InterruptedException ie) {
}

thd.stopThread();

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


#### Caution 
Use volatile only where visibility is an issue. also, you can only use this reserved word in the context of field declarations (you’ll receive an error if you try to make a local variable volatile). 

Finally, you can declare double and long fields volatile, but should avoid doing so on 32-bit JVMs because it takes two operations to access a double or long variable’s value, and mutual exclusion (via synchronized) is required to access their values safely.

#### Creating an Immutable and Thread-Safe Class with Help from final

In [11]:
import java.util.Set;
import java.util.TreeSet;

public final class Planets 
{
   private final Set<String> planets = new TreeSet<>();

   public Planets() 
   {
      planets.add("Mercury");
      planets.add("Venus");
      planets.add("Earth");
      planets.add("Mars");
      planets.add("Jupiter");
      planets.add("Saturn");
      planets.add("Uranus");
      planets.add("Neptune");
   }

   public boolean isPlanet(String planetName) 
   {
      return planets.contains(planetName);
   }
}

In [12]:
Planets planets = new Planets();
System.out.println(planets.isPlanet("Earth"));
System.out.println(planets.isPlanet("Vulcan"));

true
false


### Summary
Developing multithreaded applications is much easier when threads don’t interact, typically via shared variables. When interaction occurs, race conditions, data races, and cached variable problems can arise that make an application thread-unsafe.

You can use synchronization to solve race conditions, data races, and cached variable problems. Synchronization is a JVM feature that ensures that two or more concurrent threads don’t simultaneously execute a critical section that must be accessed in a serial manner.

Liveness refers to something beneficial happening eventually. A liveness failure occurs when an application reaches a state in which it can make no further progress. Multithreaded applications face the liveness challenges of deadlock, livelock, and starvation.
Synchronization exhibits two properties: mutual exclusion and visibility. The synchronized keyword is associated with both properties. Java also provides a weaker form of synchronization involving visibility only, and associates only this property with the volatile keyword.

When a field variable is declared volatile, it cannot also be declared final. However, this isn’t a problem because Java also lets you safely access a final field without the need for synchronization. You will often use final to help ensure thread safety in the context of an immutable class.

Chapter 3 presents waiting and notification.